In [1]:
import os
import requests
import argparse
import csv
import pathlib 
import time
from datetime import datetime
from bs4 import BeautifulSoup


In [12]:

# Funcions per obtindre la llista del indexs del Ibex 35 de la borsa de Barcelona

# Get Header
def getHeaders():   
    headers = {"Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
               "Accept-Encoding: gzip, deflate, br",               
               "Accept-Language: es-ES,es;q=0.8,en-US;q=0.5,en;q=0.3",  
               "Connection: keep-alive",                              
               "TE: Trailers",
               "Upgrade-Insecure-Requests: 1",
               "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0"}
    return headers

        
# Get table Page        
def getPageTable(UrlPage):        
    tabla = None
    try:
        # Data for post form    
        data = {'punto':'indice'}            
        page = requests.post(UrlPage,timeout=10)    
        soup = BeautifulSoup(page.text, "html.parser")
        tabla = soup.find('table', attrs={'id': 'ctl00_Contenido_tblAcciones'})                   
    except requests.exceptions.Timeout:
        print("Timeout a la càrrega de la web")        
    except:        
        print("Error inesperat:", UrlPage)
    return tabla        

# Get Firm page and wait x seconds
def getFirmTablePage(UrlFirmPage):    
    table = None    
    try:
        t0 = time.time()
        # Data for post form            
        data = {'punto':'indice'}    
        page = requests.get(UrlFirmPage,timeout=10)                    
        soup = BeautifulSoup(page.text, "html.parser")        
        print("->Link:"+ UrlFirmPage)        
        tabla = soup.find('table', attrs={'id': 'ctl00_Contenido_tblCapital'})                           
        # Wait time depending of response
        response_delay = time.time() - t0     
        time.sleep(10 * response_delay)
    except requests.exceptions.Timeout:
        print("Timeout a la càrrega de dada de la companyia")        
        print("--->No Referencia timeout:"+ UrlFirmPage)
    except:                
        print("--->No Referencia Error inesperado :"+ UrlFirmPage)
    return table

# Get the list of firm interesting values
def getFirmValues(UrlBase,firmPage):
    Url = UrlBase+firmPage        
    values = []
    table = getFirmTablePage(Url)    
    if (table is not None):        
        nroFila=0
        for row in tabla.find_all("tr"):
            if rowNum==2:
                colNum=0
                for cell in row.find_all('td'):
                    value=["",""]
                    if colNum==2:
                        value=[cell.text,""]              
                        values.append(value)    
                        print("Capit.", value[0])            
                        colNum=colNum+1
            rowNum=rowNum+1       
    return values    
        

# Set List with table page values
def getIndexs(UrlBase,UrlRef):
    indexList = []
    UrlIndexPage = UrlBase+UrlRef
    table = getPageTable(UrlIndexPage)    
    # Extract table values and links
    rowList = []
    for tr in table.find_all("tr"):        
        colList = []               
        valueFirm = []
        # Cells Header
        cellsHead = tr.findAll('th')                
        if ((cellsHead is not None) and (len(cellsHead) > 0)):
             for cellHead in cellsHead:        
                    value = [cellHead.text.strip(),'']                  
                    colList.append(value)                                                   
        else:
            # Cells value
            cells = tr.findAll('td')                        
            for cell in cells:
                href = ''
                infComp = []
                a= cell.find('a')                  
                if (a is not None):                 
                    href = a.get('href')                                
                    # Append / if needed
                    if href[0] !='/':
                        href = '/' + href   
                    print(cell.text.strip())
                    valueFirm = getFirmValues(UrlBase,href)                     
                value = [cell.text.strip(),href]                                  
                colList.append(value)             
                
        # Extend list with firm information 
        colList.extend(valueFirm)
        # Append to rowlist    
        rowList.append(colList)                 
    return rowList 


def writecsv(table,filename):
    currentDir = os.path.abspath('')
    filePath = os.path.join(currentDir,filename)
    file = open(filePath,"w+")
    for row in table:
        for col in row:
            file.write(col[0]+";")
            file.write("\n")
    

In [13]:
# indicar la ruta
urlbase = 'https://www.borsabcn.es'
urlRef= '/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000&punto=indice'
indexList = getIndexs(urlbase,urlRef);


ACCIONA
--->Referencia:https://www.borsabcn.es/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0125220311
ACERINOX
--->Referencia:https://www.borsabcn.es/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0132105018
ACS
--->Referencia:https://www.borsabcn.es/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0167050915
AENA
--->Referencia:https://www.borsabcn.es/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0105046009
ALMIRALL
--->Referencia:https://www.borsabcn.es/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0157097017
AMADEUS
--->Referencia:https://www.borsabcn.es/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0109067019
ARCELORMIT.
--->Referencia:https://www.borsabcn.es/esp/aspx/Empresas/FichaValor.aspx?ISIN=LU1598757687
B.SANTANDER
--->Referencia:https://www.borsabcn.es/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0113900J37
BA.SABADELL
--->Referencia:https://www.borsabcn.es/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0113860A34
BANKINTER
--->Referencia:https://www.borsabcn.es/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0113679I37
B

In [ ]:
writecsv(indexList,"ListaIndices.csv")